In [8]:
# testing collab feature - marija
import ee
ee.Initialize(project='gist-itproject2')  

#NEW MESSAGE 
#CAN YOU SEE THIS
#omg yes this is so cool 
#better than word! also does running the code work 

In [15]:
import ee, geemap
import zipfile
import os
import json
from ipyleaflet import Marker, DivIcon, Popup, CircleMarker, LayerGroup
from ipywidgets import HTML, Box
# --- EE init (edit project if needed) ---
ee.Initialize(project='gist-itproject2')

# -------------------------------
# 1) Indo‑Pacific country set
# (LSIB_2017 'country_na' names)
# -------------------------------
ip_names = [
    #Countries here are highlighted in yellow
    # South Asia
    "Afghanistan", "Bangladesh", "Bhutan", "India", "Maldives",
    "Nepal", "Pakistan", "Sri Lanka",

    # Southeast Asia
    "Brunei", "Burma",  # LSIB uses "Burma" instead of "Myanmar"
    "Cambodia", "Indonesia", "Laos", "Malaysia", "Philippines",
    "Singapore", "Thailand", "Timor-Leste", "Vietnam",

    # East Asia
    "China", "Hong Kong", "Macau", "Japan", "Korea, South", "Korea, North", "Mongolia", "Taiwan",

    # Oceania
    "Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia, Federated States of",
    "Nauru", "New Zealand", "Palau", "Papua New Guinea", "Samoa",
    "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu"
]

# -------------------------------
# 2) Profiles (only some countries have data yet)
#    Keys must match LSIB names above
# -------------------------------
profiles = {
    "Australia": {
        "Capital":"Canberra","Ruling":"Australian Labor Party",
        "Opposition":"Liberal‑Nationals Coalition",
        "Religion1":"No Religion (38.7%)","Religion2":"Catholic (19.6%)"
    },
    "China": {
        "Capital":"Beijing","Ruling":"Chinese Communist Party (CCP)",
        "Opposition":"None (one‑party state)",
        "Religion1":"Unaffiliated (52.1%)","Religion2":"Chinese Folk Religion (21.9%)"
    },
    "India": {
        "Capital":"New Delhi","Ruling":"BJP‑led coalition","Opposition":"INDIA bloc",
        "Religion1":"Hinduism (79.8%)","Religion2":"Islam (14.2%)"
    },
    "Indonesia": {
        "Capital":"Nusantara (transitional)","Ruling":"Gerindra (President)","Opposition":"PDI‑P (role in flux)",
        "Religion1":"Muslim (87.2%)","Religion2":"Protestant (7%)"
    },
    "Japan": {
        "Capital":"Tokyo","Ruling":"LDP–Komeito coalition","Opposition":"CDP",
        "Religion1":"Shinto (48.6%)","Religion2":"Buddhism (46.4%)"
    },
    "Malaysia": {
        "Capital":"Kuala Lumpur","Ruling":"Pakatan Harapan (PH) coalition","Opposition":"Perikatan Nasional (PN)",
        "Religion1":"Islam (63.5%)","Religion2":"Buddhism (18.7%)"
    },
    "New Zealand": {
        "Capital":"Wellington","Ruling":"National‑led coalition","Opposition":"Labour Party",
        "Religion1":"No Religion (51.6%)","Religion2":"Christianity (32.3%)"
    },
    "Philippines": {
        "Capital":"Manila","Ruling":"Alyansa para sa Bagong Pilipinas",
        "Opposition":"DuterTen coalition",
        "Religion1":"Roman Catholic (78.8%)","Religion2":"Islam (6.4%)"
    },
    "Korea, South": {
        "Capital":"Seoul","Ruling":"People Power Party (PPP)",
        "Opposition":"Democratic Party (DP)",
        "Religion1":"Unaffiliated (46.5%)","Religion2":"Christian (29%)"
    },
    "Thailand": {
        "Capital":"Bangkok","Ruling":"Pheu Thai coalition",
        "Opposition":"People's Party (formerly MFP)",
        "Religion1":"Theravada Buddhism (90–95%)","Religion2":"Muslim (~5%)"
    },
    "Vietnam": {
        "Capital":"Hanoi","Ruling":"Communist Party of Vietnam (CPV)",
        "Opposition":"None (one‑party)",
        "Religion1":"Irreligion/Folk Religion (76.5%)","Religion2":"Buddhism (13.3%)"
    }
}

# -------------------------------
# 3) Build layers (clean + fast)
# -------------------------------
world = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Indo‑Pacific polygons by explicit name list (no extent guesswork)
ip_fc = world.filter(ee.Filter.inList('country_na', ip_names))
non_ip_fc = world.filter(ee.Filter.Not(ee.Filter.inList('country_na', ip_names)))

# Attach profiles where available; otherwise blank strings
def set_profile(f):
    name = ee.String(f.get('country_na'))
    prof = ee.Dictionary(profiles).get(name, ee.Dictionary({}))
    prof = ee.Dictionary(prof)
    return f.set({
        'Country': name,
        'Capital': prof.get('Capital', ''),
        'RulingParty': prof.get('Ruling', ''),
        'Opposition': prof.get('Opposition', ''),
        'Religion1': prof.get('Religion1', ''),
        'Religion2': prof.get('Religion2', '')
    })

ip_fc = ip_fc.map(set_profile)

# One diamond per country: take centroid of the first feature per country
centroids = (
    ip_fc
    .map(lambda f: ee.Feature(f.geometry().centroid(1), f.toDictionary()))
    .distinct('Country')   # remove duplicates by country
)

# -------------------------------
# 4) Styling & map
# -------------------------------
m = geemap.Map(center=[-10, 140], zoom=3)
m.add_basemap("CartoDB.Positron")

# Beautified styles
non_ip_style = non_ip_fc.style(color="d0d0d0", fillColor="5ca2ff", width=0.2)
ip_fill      = ip_fc.style(color="ffffff", fillColor="ffcc00", width=1.1)   # yellow fill
ip_outline   = ip_fc.style(color="ffffff", fillColor="00000000", width=2.0) # crisp border

# add blue below, then yellow, then crisp outline
m.addLayer(non_ip_style, {}, "Outside Indo‑Pacific", True, 0.18)
m.addLayer(ip_fill, {}, "Indo‑Pacific (fill)", True, 0.95)
m.addLayer(ip_outline, {}, "Indo‑Pacific (outline)", True, 1.0)

# -------------------------------
# 5) Diamond markers + HTML popups
# -------------------------------
gj = geemap.ee_to_geojson(centroids)

diamond_html = """
<div style="
  width:14px;height:14px;background:#ffcc00;border:2px solid white;
  transform: rotate(45deg);box-shadow:0 0 3px rgba(0,0,0,.35);
  border-radius:1px;
"></div>"""

# Create a group for all diamond markers
diamond_group = LayerGroup(name="Country Polygons")

for feat in gj["features"]:
    lon, lat = feat["geometry"]["coordinates"]
    p = feat["properties"]
    
    popup_content = HTML(value=f"""
      <div style="font-family:system-ui;min-width:240px;line-height:1.35">
        <div style="font-weight:700;margin-bottom:4px">{p.get('Country','')}</div>
        <div><b>Capital:</b> {p.get('Capital','') or '—'}</div>
        <div><b>Ruling:</b> {p.get('RulingParty','') or '—'}</div>
        <div><b>Opposition:</b> {p.get('Opposition','') or '—'}</div>
        <div><b>Religions:</b> {p.get('Religion1','') or '—'}{', ' if p.get('Religion2') else ''}{p.get('Religion2','')}</div>
      </div>
    """)
    
    popup = Popup(location=(lat, lon), child=popup_content, max_width=280)
    marker = Marker(location=(lat, lon), icon=DivIcon(html=diamond_html), draggable=False)
    marker.popup = popup
    
    diamond_group.add_layer(marker)

m.add_layer(diamond_group)

# Add a message about the markers
print("Note: Hover over markers to see country names. Full popup functionality may require additional configuration.")
# -------------------------------
# 6) Add Tectonic Plate Boundaries (from shapefile)
# -------------------------------

# Step 1: Define paths
zip_path = '2 - plate_boundaries_edit.zip'
extract_dir = './plate_boundaries'  # Writable local folder

# Step 2: Unzip the shapefile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Step 3: Find the .shp file
shapefile = [f for f in os.listdir(extract_dir) if f.endswith('.shp')][0]
shapefile_path = os.path.join(extract_dir, shapefile)

# Step 4: Try different approaches to add the data to the map
# Use the built-in add_shapefile method of geemap
m.add_shapefile(shapefile_path, layer_name="Tectonic Plate Boundaries", style={"color": "orange", "weight": 2})
print("Shapefile Loaded directly to map.")
m

Note: Hover over markers to see country names. Full popup functionality may require additional configuration.
Shapefile Loaded directly to map.


Map(center=[-10, 140], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [11]:
#m.to_html("MainMap.html")